# Sample on the flux space of the metabolic network of e_coli

### Use library sybil to read e_coli data and run FBA algorithm

In [1]:
suppressMessages(library(sybil))
options(warn=-1)

mp <- system.file(package = "sybil", "extdata")
mod <- readTSVmod(prefix = "Ec_core", fpath = mp, quoteChar = "\"")
modelorg2tsv(mod, prefix = "Ec_core")
data(Ec_core)
ex <- findExchReact(Ec_core)
optL <- optimizeProb(Ec_core, algorithm = "fba", retOptSol = FALSE)
fba <- optimizeProb(Ec_core, algorithm = "fba")

reading model description, ... OK
reading metabolite list ... OK
parsing reaction list ... OK
GPR mapping ... OK
sub systems ... OK
prepare modelorg object ... OK
validating object ... OK


[1] TRUE

Loading required package: glpkAPI
using GLPK version 4.47


### Extract all constrants in order to define flux space

1. $S$ :  stoichiometric matrix

2. $low\_bound$ : capacity's lower bound

3. $up\_bound$ : capacity's upper bound

In [2]:
S<- (as.matrix(S(Ec_core)))

low_bound<-mod@lowbnd
up_bound<-mod@uppbnd

### Define constrants inequetions

#### Two different constraints make the flux space:

1. $Sv=0$, where $S$ is the stoichiometric matrix and $v$ a unknown vector we are looking for.
2. $a_i\leq v_i\leq b_i$, where $a_i$ defined by $low\_bound$ and $b_i$ by $up\_bound$

In oder to represent these constraints we consider the linear inequality $Ax\leq b$. So we construct $A$ matrix and $b$ vector.

In [3]:
m = dim(S)[1]
n = dim(S)[2]

up_matrix = diag(n)
low_matrix2 = -1*diag(n)

S2 = -1*S
 
zero_con = numeric(m)+10 #NOTE:                        ***Future Changes**
                         #Inequality contraints we contruct have only one fisible solution equal to zero vector.
                         #      In order to show the whole sampling proccess we change constraint (1) 

A<-rbind(up_matrix,low_matrix2,S,S2)
b<-rbind(t(matrix(up_bound,nrow = 1,ncol = n)),
         t(matrix(-1*low_bound,nrow = 1,ncol = n)),
         t(matrix(zero_con,nrow = 1,ncol = m)),
         t(matrix(zero_con,nrow = 1,ncol = m)))

### Polytope's Construction and Sampling

#### A Convex polytope can be described as  $Ax\leq b$ linear inequality.

In [4]:
suppressMessages(library(volesti))
P = Hpolytope(A = A, b = c(b))
sampled_data<-sample_points(P, 10000)

### Save sampled data as a csv file

In [5]:
suppressMessages(library(MASS))
current_path = 'C:/Users/petsi/Documents/UOA/Google'
name = '/sampled_data.csv'
full_path=paste(current_path,name,sep='')
write.matrix(sampled_data, file = full_path, sep = ",")